In [2]:
pip install matplotlib numpy

In [3]:
!unzip "/content/drive/MyDrive/GAN/archive (2).zip"


Archive:  /content/drive/MyDrive/GAN/archive (2).zip
  inflating: lol_dataset/eval15/high/1.png  
  inflating: lol_dataset/eval15/high/111.png  
  inflating: lol_dataset/eval15/high/146.png  
  inflating: lol_dataset/eval15/high/179.png  
  inflating: lol_dataset/eval15/high/22.png  
  inflating: lol_dataset/eval15/high/23.png  
  inflating: lol_dataset/eval15/high/493.png  
  inflating: lol_dataset/eval15/high/547.png  
  inflating: lol_dataset/eval15/high/55.png  
  inflating: lol_dataset/eval15/high/665.png  
  inflating: lol_dataset/eval15/high/669.png  
  inflating: lol_dataset/eval15/high/748.png  
  inflating: lol_dataset/eval15/high/778.png  
  inflating: lol_dataset/eval15/high/780.png  
  inflating: lol_dataset/eval15/high/79.png  
  inflating: lol_dataset/eval15/low/1.png  
  inflating: lol_dataset/eval15/low/111.png  
  inflating: lol_dataset/eval15/low/146.png  
  inflating: lol_dataset/eval15/low/179.png  
  inflating: lol_dataset/eval15/low/22.png  
  inflating: lol_data

In [4]:
!ls

'archive (2).zip'   drive   lol_dataset   sample_data


In [5]:
!pip install tensorflow numpy pillow scikit-image torch torchvision lpips

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 65.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 56.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 4.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
from PIL import Image
import os
import glob
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
import torch
import lpips
import torchvision.transforms as transforms

# Define the Pix2Pix Generator (U-Net architecture)
def build_pix2pix_generator():
    inputs = layers.Input(shape=(256, 256, 3))
    down_stack = [
        layers.Conv2D(64, 4, strides=2, padding='same', activation='relu'),
        layers.Conv2D(128, 4, strides=2, padding='same', activation='relu'),
        layers.Conv2D(256, 4, strides=2, padding='same', activation='relu'),
        layers.Conv2D(512, 4, strides=2, padding='same', activation='relu'),
        layers.Conv2D(512, 4, strides=2, padding='same', activation='relu'),
        layers.Conv2D(512, 4, strides=2, padding='same', activation='relu'),
        layers.Conv2D(512, 4, strides=2, padding='same', activation='relu'),
        layers.Conv2D(512, 4, strides=2, padding='same', activation='relu'),
    ]
    up_stack = [
        layers.Conv2DTranspose(512, 4, strides=2, padding='same', activation='relu'),
        layers.Conv2DTranspose(512, 4, strides=2, padding='same', activation='relu'),
        layers.Conv2DTranspose(512, 4, strides=2, padding='same', activation='relu'),
        layers.Conv2DTranspose(256, 4, strides=2, padding='same', activation='relu'),
        layers.Conv2DTranspose(128, 4, strides=2, padding='same', activation='relu'),
        layers.Conv2DTranspose(64, 4, strides=2, padding='same', activation='relu'),
        layers.Conv2DTranspose(32, 4, strides=2, padding='same', activation='relu'),
    ]
    x = inputs
    skips = []
    for down in down_stack:
        x = down(x)
        skips.append(x)
    skips = reversed(skips[:-1])
    for up, skip in zip(up_stack, skips):
        x = up(x)
        x = layers.Concatenate()([x, skip])
    outputs = layers.Conv2DTranspose(3, 4, strides=2, padding='same', activation='tanh')(x)
    return models.Model(inputs, outputs, name='pix2pix_generator')

# Define the Pix2Pix Discriminator (PatchGAN)
def build_pix2pix_discriminator():
    inputs = layers.Input(shape=(256, 256, 3))
    target = layers.Input(shape=(256, 256, 3))
    x = layers.Concatenate()([inputs, target])
    x = layers.Conv2D(64, 4, strides=2, padding='same', activation='relu')(x)
    x = layers.Conv2D(128, 4, strides=2, padding='same', activation='relu')(x)
    x = layers.Conv2D(256, 4, strides=2, padding='same', activation='relu')(x)
    x = layers.Conv2D(512, 4, strides=2, padding='same', activation='relu')(x)
    x = layers.Conv2D(1, 4, padding='same', activation='sigmoid')(x)
    return models.Model([inputs, target], x, name='pix2pix_discriminator')

# Load and preprocess image pairs from the LOL dataset
def load_image_pair(low_path, high_path):
    low_image = Image.open(low_path).convert('RGB')
    high_image = Image.open(high_path).convert('RGB')
    low_image = low_image.resize((256, 256))
    high_image = high_image.resize((256, 256))
    low_image = np.array(low_image).astype(np.float32) / 127.5 - 1
    high_image = np.array(high_image).astype(np.float32) / 127.5 - 1
    return low_image, high_image

# Simulate lighting effects (D75, D65, night light) on enhanced images
def apply_lighting_effect(image, temperature, brightness_factor, blue_boost=1.0):
    img_array = (image + 1) * 127.5  # Denormalize to [0, 255]
    img_array = img_array.astype(float)

    if temperature > 6500:
        blue_boost_factor = (temperature - 6500) / 10000
        img_array[:, :, 2] = img_array[:, :, 2] * (1 + blue_boost_factor)
        img_array[:, :, 0] = img_array[:, :, 0] * (1 - blue_boost_factor * 0.5)
    elif temperature < 6500:
        yellow_boost = (6500 - temperature) / 10000
        img_array[:, :, 0] = img_array[:, :, 0] * (1 + yellow_boost)
        img_array[:, :, 1] = img_array[:, :, 1] * (1 + yellow_boost * 0.5)
        img_array[:, :, 2] = img_array[:, :, 2] * (1 - yellow_boost * 0.5)

    img_array = img_array * brightness_factor
    img_array = np.clip(img_array, 0, 255)

    if blue_boost != 1.0:
        img_array[:, :, 2] = img_array[:, :, 2] * blue_boost

    img_array = img_array.astype(np.float32) / 127.5 - 1  # Re-normalize to [-1, 1]
    return img_array

# Loss functions
def generator_loss(disc_output, gen_output, target, lambda_l1=100):
    gan_loss = tf.reduce_mean(tf.keras.losses.binary_crossentropy(tf.ones_like(disc_output), disc_output))
    l1_loss = tf.reduce_mean(tf.abs(target - gen_output))
    return gan_loss + lambda_l1 * l1_loss

def discriminator_loss(real_output, fake_output):
    real_loss = tf.reduce_mean(tf.keras.losses.binary_crossentropy(tf.ones_like(real_output), real_output))
    fake_loss = tf.reduce_mean(tf.keras.losses.binary_crossentropy(tf.zeros_like(fake_output), fake_output))
    return real_loss + fake_loss

# Training step
@tf.function
def train_step(input_image, target, generator, discriminator, gen_optimizer, disc_optimizer):
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        gen_output = generator(input_image, training=True)
        disc_real_output = discriminator([input_image, target], training=True)
        disc_fake_output = discriminator([input_image, gen_output], training=True)
        gen_loss = generator_loss(disc_fake_output, gen_output, target)
        disc_loss = discriminator_loss(disc_real_output, disc_fake_output)
    gen_grads = gen_tape.gradient(gen_loss, generator.trainable_variables)
    disc_grads = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
    gen_optimizer.apply_gradients(zip(gen_grads, generator.trainable_variables))
    disc_optimizer.apply_gradients(zip(disc_grads, discriminator.trainable_variables))
    return gen_loss, disc_loss

# Evaluation function
def evaluate_images(target, generated):
    target = (target + 1) * 127.5
    generated = (generated + 1) * 127.5
    target = np.clip(target, 0, 255).astype(np.uint8)
    generated = np.clip(generated, 0, 255).astype(np.uint8)
    psnr_value = psnr(target, generated, data_range=255)
    ssim_value = ssim(target, generated, channel_axis=2, data_range=255)
    lpips_model = lpips.LPIPS(net='alex')
    target_tensor = torch.from_numpy(target).permute(2, 0, 1).float() / 255.0
    generated_tensor = torch.from_numpy(generated).permute(2, 0, 1).float() / 255.0
    target_tensor = target_tensor.unsqueeze(0) * 2 - 1
    generated_tensor = generated_tensor.unsqueeze(0) * 2 - 1
    lpips_value = lpips_model(target_tensor, generated_tensor).item()
    return psnr_value, ssim_value, lpips_value

# Load the LOL dataset
def load_lol_dataset(dataset_path='/content/lol_dataset'):
    train_low_dir = os.path.join(dataset_path, 'our485/low')
    train_high_dir = os.path.join(dataset_path, 'our485/high')
    test_low_dir = os.path.join(dataset_path, 'eval15/low')
    test_high_dir = os.path.join(dataset_path, 'eval15/high')

    train_low_files = sorted(glob.glob(os.path.join(train_low_dir, '*.png')))
    train_high_files = sorted(glob.glob(os.path.join(train_high_dir, '*.png')))
    test_low_files = sorted(glob.glob(os.path.join(test_low_dir, '*.png')))
    test_high_files = sorted(glob.glob(os.path.join(test_high_dir, '*.png')))

    # Load training data
    train_inputs, train_targets = [], []
    for low_file, high_file in zip(train_low_files, train_high_files):
        low_img, high_img = load_image_pair(low_file, high_file)
        train_inputs.append(low_img)
        train_targets.append(high_img)

    # Load test data
    test_inputs, test_targets = [], []
    for low_file, high_file in zip(test_low_files, test_high_files):
        low_img, high_img = load_image_pair(low_file, high_file)
        test_inputs.append(low_img)
        test_targets.append(high_img)

    return (np.array(train_inputs), np.array(train_targets)), (np.array(test_inputs), np.array(test_targets))

# Main script
dataset_path = '/content/lol_dataset'
(train_inputs, train_targets), (test_inputs, test_targets) = load_lol_dataset(dataset_path)

# Build Pix2Pix models
generator = build_pix2pix_generator()
discriminator = build_pix2pix_discriminator()

# Optimizers
gen_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
disc_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

# Training loop
epochs = 10  # Adjust based on your needs
batch_size = 1  # Small batch size due to limited data
for epoch in range(epochs):
    for i in range(0, len(train_inputs), batch_size):
        batch_inputs = train_inputs[i:i+batch_size]
        batch_targets = train_targets[i:i+batch_size]
        gen_loss, disc_loss = train_step(batch_inputs, batch_targets, generator, discriminator, gen_optimizer, disc_optimizer)
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch + 1}, Gen Loss: {gen_loss:.4f}, Disc Loss: {disc_loss:.4f}")

# Evaluate on test set and apply lighting effects
conditions = ['d75', 'd65', 'night_light']
lighting_params = [
    (7500, 1.2, 1.0),   # D75: Cooler, brighter
    (6500, 0.5, 1.0),   # D65: Neutral, dusk-like
    (4100, 0.3, 1.1),   # Night light: Dim, moonlight
]
results = {'PSNR': [], 'SSIM': [], 'LPIPS': []}

# Create output directory
os.makedirs('output', exist_ok=True)

for i in range(min(5, len(test_inputs))):  # Evaluate on first 5 test images
    input_img = tf.expand_dims(test_inputs[i], 0)
    target_img = test_targets[i]

    # Enhance low-light image to normal light
    enhanced_img = generator(input_img, training=False)[0].numpy()

    # Apply lighting effects
    for condition, (temp, brightness, blue_boost) in zip(conditions, lighting_params):
        gen_output = apply_lighting_effect(enhanced_img, temp, brightness, blue_boost)

        # Save the generated image
        gen_img = (gen_output + 1) * 127.5
        gen_img = np.clip(gen_img, 0, 255).astype(np.uint8)
        Image.fromarray(gen_img).save(f'output/test_{i}_{condition}.jpg')

        # Evaluate against the original normal-light image (target)
        psnr_value, ssim_value, lpips_value = evaluate_images(target_img, gen_output)
        results['PSNR'].append(psnr_value)
        results['SSIM'].append(ssim_value)
        results['LPIPS'].append(lpips_value)
        print(f"\nEvaluation for test image {i}, condition {condition}:")
        print(f"PSNR: {psnr_value:.2f} dB")
        print(f"SSIM: {ssim_value:.4f}")
        print(f"LPIPS: {lpips_value:.4f}")

# Print average metrics
print("\nAverage Metrics Across All Test Images and Conditions:")
print(f"Average PSNR: {np.mean(results['PSNR']):.2f} dB")
print(f"Average SSIM: {np.mean(results['SSIM']):.4f}")
print(f"Average LPIPS: {np.mean(results['LPIPS']):.4f}")

print("\nGenerated images saved in 'output' directory.")

Epoch 10, Gen Loss: 47.8121, Disc Loss: 0.6520
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:04<00:00, 51.0MB/s]


Loading model from: /usr/local/lib/python3.11/dist-packages/lpips/weights/v0.1/alex.pth

Evaluation for test image 0, condition d75:
PSNR: 12.68 dB
SSIM: 0.3494
LPIPS: 0.5622
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.11/dist-packages/lpips/weights/v0.1/alex.pth

Evaluation for test image 0, condition d65:
PSNR: 8.74 dB
SSIM: 0.3419
LPIPS: 0.6161
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.11/dist-packages/lpips/weights/v0.1/alex.pth

Evaluation for test image 0, condition night_light:
PSNR: 7.34 dB
SSIM: 0.2774
LPIPS: 0.7027
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.11/dist-packages/lpips/weights/v0.1/alex.pth

Evaluation for test image 1, condition d75:
PSNR: 16.69 dB
SSIM: 0.4705
LPIPS: 0.6121
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loadin